## Example Grid Search Signal 1.1: VIX Return via News Sentiment

In [ ]:
### Window Generator
set_seed(123)

# sliding window & look back
lf = 2  # 52     # look forward (weeks), how far to predict the target variable (variables in lab_columns below)
ks = 1  # 26     # kernel size   26 (larger kernel allows more general/global patterns), 26 time steps considered at once
lw = 1  # 1      # label width (weeks), how many weeks model predicts at once -> e.g. lf=52 lw=1 -> 52 times 1 week prediction
lb = 1  # 52     # look back (weeks), how many weeks to look back to for inp variables (inp_column)
pooling = 1

# Defining input and target variables
inp_columns = ['NewsSent'] # input
lab_columns=['VIX'] # target that we want to predict

window = WindowGenerator(input_width=lb, label_width=lw, shift=lf, input_columns=inp_columns, label_columns=lab_columns, all_columns=train_df.columns)
td = window.make_dataset(train_df, shuffle=True)

# inp shape: (lb, ks, num_variables)

# cross-validation
is_data = td.take(2)
os_data = td.skip(2)


!pip install -Uq wandb

import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint, WandbCallback

wandb.login()

### Training
# Hint: play with num_hidden = 1 or 2, and kernel_size

def directional_accuracy(y_true, y_pred):
    """
    Accuracy based on whether correct direction is predicted
    """
    y_true_direction = tf.sign(y_true[1:] - y_true[:-1])
    y_pred_direction = tf.sign(y_pred[1:] - y_pred[:-1])
    correct_predictions = tf.reduce_mean(tf.cast(tf.equal(y_true_direction, y_pred_direction), dtype=tf.float32))
    return correct_predictions


# learning rate schedule
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    0.01, # initial learning rate
    decay_steps=50,
    decay_rate=0.97,
    staircase=True
    )


# define early stopping condition
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=50, mode='min')

sweep_config = {
    'method': 'grid',
    'metric': {
        'name': 'val_loss',
        'goal': 'minimize'
    },
    'parameters': {
        'dropout_rate': {
            'values': [0.2, 0.3, 0.4]
        },
        'num_epochs': {
            'values': [10, 20, 30]
        },
        'regularization': {
            'values': [0.01, 0.001, 0.0001]
        },
        'batch_size': {
            'values': [16, 32, 64]
        }
    }
}

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    0.001, # initial learning rate
    decay_steps=50,
    decay_rate=0.97,
    staircase=True
    )


def train_model():
    with wandb.init() as run:
        config = wandb.config

        model = create_cnn_extended(ks, lb, lw, config.dropout_rate, config.regularization)

        model.compile(
            loss=tf.losses.MeanSquaredError(),
            optimizer=tf.optimizers.Adam(learning_rate=lr_schedule),
            metrics=[tf.metrics.MeanSquaredError(), directional_accuracy]
        )

        history = model.fit(
            is_data,
            validation_data=os_data,
            epochs=config.num_epochs,
            batch_size=config.batch_size,
            callbacks=[early_stopping, WandbCallback()]
        )
        return history

# Run the wandb sweep
sweep_id = wandb.sweep(sweep_config, project="bf_proj")
wandb.agent(sweep_id, train_model)

model.summary()

